In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import torch
from torch import nn
from torch.optim import Adam
from transformers import BertModel, BertConfig
from tqdm import tqdm

#Import model for evaluation
model = torch.load('ROUTE TO MODEL')

In [ ]:
#Model evaluation function
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
#Model prediction function
def predict(model, test_data):

    test = PredictDataset(test_data)

    dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()
    
    outputs = []
    with torch.no_grad():
        for test_input in dataloader:
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              label = id2label[output.argmax(dim=1).item()]
              outputs.append(label)
    return outputs